In [2]:
#%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import datasets, layers, models, callbacks
  
print(tf.version)

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))


# Helper libraries
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import scipy.io as sio
from sklearn.model_selection import train_test_split
from sklearn.utils import compute_class_weight

from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from pathlib import Path
import os.path
#import datetime, os

from keras.applications import VGG16, VGG19, ResNet50, InceptionV3, InceptionResNetV2, Xception, MobileNet, DenseNet121, \
    NASNetMobile, EfficientNetB0, MobileNetV2, MobileNetV3Large
from tensorflow.python.keras.layers import Conv2D, Flatten, Dropout, Dense, MaxPooling2D


<module 'tensorflow._api.v2.version' from '/home/jarrad_foley135/miniconda3/envs/mlis_cw_car/lib/python3.9/site-packages/tensorflow/_api/v2/version/__init__.py'>
Num GPUs Available:  1


In [3]:
batch_size = 16
img_size = [240, 320]

# df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
# df['filename'] = df["image_id"].astype(str) + ".png"

# df = df.sample(frac=1).reset_index(drop=True)
# print(df)

#df.to_csv("training_norm_shuffle.csv", encoding='utf-8', index=False)

df = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm_shuffle.csv')
print(df)

x_train = df[0:int(len(df) * 0.6)]
x_validate = df[int(len(df) * 0.6):int(len(df) * 0.8)]
x_evaluate = df[int(len(df) * 0.8):]

# x_train = df[0:int(len(df) * 0.8)]
# x_validate = df[int(len(df) * 0.8):]


       image_id   angle  speed   filename
0           414  0.8125    1.0    414.png
1         17532  0.6875    1.0  17532.png
2         16480  0.3125    0.0  16480.png
3          6450  0.6875    0.0   6450.png
4         12314  0.6250    0.0  12314.png
...         ...     ...    ...        ...
14821      9844  0.6875    1.0   9844.png
14822     12367  0.5000    1.0  12367.png
14823       343  0.9375    1.0    343.png
14824      3838  0.6875    0.0   3838.png
14825     13301  0.5000    1.0  13301.png

[14826 rows x 4 columns]


In [4]:

shift = 0.1

training_datagen = ImageDataGenerator(
    rescale = 1./255,
    fill_mode='nearest',
    width_shift_range=shift,
    height_shift_range=shift
)

train_generator = training_datagen.flow_from_dataframe(
    dataframe=x_train,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    batch_size=batch_size,
    shuffle=False,
    class_mode='other')

#print(train_generator)

validation_datagen = ImageDataGenerator(rescale = 1./255)

val_generator = validation_datagen.flow_from_dataframe(
    dataframe=x_validate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')

evaluate_datagen = ImageDataGenerator(rescale = 1./255)

eval_generator = evaluate_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    y_col=["angle","speed"],
    target_size=img_size,
    shuffle=False,
    class_mode='other')


Found 8895 validated image filenames.
Found 2965 validated image filenames.
Found 2966 validated image filenames.


In [4]:
shape = (*img_size, 3) # inherited image size with 3 color filters
input_shape = [240, 320, 3]
transfer = MobileNetV2(
    input_shape=input_shape, 
    include_top=False, 
    weights='imagenet',
    classifier_activation="relu"
)

for layer in transfer.layers[:-2]:
    layer.trainable = False

# CNN base
model = models.Sequential()

# model.add(layers.Conv2D(16, (3, 3), activation='relu', input_shape=shape))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2)))

model.add(transfer)
model.build()
        
model.add(layers.Conv2D(128, (3, 3), strides= (2,2), activation='relu'))
model.add(layers.BatchNormalization())

# model.add(layers.Conv2D(64, (3, 3), activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.MaxPooling2D((2, 2), padding='same'))

model.add(layers.Flatten()),

model.add(layers.Dense(64, activation='relu'))
model.add(layers.BatchNormalization())
model.add(layers.Dropout(0.2))

# model.add(layers.Dense(16, activation='relu'))
# model.add(layers.BatchNormalization())
# model.add(layers.Dropout(0.2))

# Output layer
model.add(layers.Flatten())
model.add(layers.Dense(2,  activation='relu', kernel_initializer='normal'))

2023-03-26 17:49:29.128611: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-26 17:49:29.132122: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-26 17:49:29.132187: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-03-26 17:49:29.132210: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:967] could not open file to read NUMA node: /sys/bus/pci/devices/0

In [5]:
model.build()


In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 10, 1280)      2257984   
 ional)                                                          
                                                                 
 conv2d (Conv2D)             (None, 3, 4, 128)         1474688   
                                                                 
 batch_normalization (BatchN  (None, 3, 4, 128)        512       
 ormalization)                                                   
                                                                 
 flatten (Flatten)           (None, 1536)              0         
                                                                 
 dense (Dense)               (None, 64)                98368     
                                                                 
 batch_normalization_1 (Batc  (None, 64)               2

In [7]:
model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.001),
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.001,
    patience=20,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

history = model.fit(
    train_generator,
    batch_size=batch_size,
    validation_data=val_generator,
    callbacks=[es],
    epochs=100
)

Epoch 1/100


2023-03-26 17:49:36.021329: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:428] Loaded cuDNN version 8100
2023-03-26 17:49:41.923241: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: Permission denied
2023-03-26 17:49:46.652172: I tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:630] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.
2023-03-26 17:49:46.685163: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x2cb44360 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-26 17:49:46.685264: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 3070 Laptop GPU, Compute Capability 8.6
2023-03-26 17:49:46.715033: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-26 17:49:47.087889

556/556 [==============================] - 120s 187ms/step - loss: 0.0915 - val_loss: 0.0264
Epoch 2/100
556/556 [==============================] - 101s 181ms/step - loss: 0.0304 - val_loss: 0.0198
Epoch 3/100
556/556 [==============================] - 100s 181ms/step - loss: 0.0233 - val_loss: 0.0154
Epoch 4/100
556/556 [==============================] - 99s 178ms/step - loss: 0.0211 - val_loss: 0.0144
Epoch 5/100
556/556 [==============================] - 99s 178ms/step - loss: 0.0188 - val_loss: 0.0135
Epoch 6/100
556/556 [==============================] - 100s 179ms/step - loss: 0.0178 - val_loss: 0.0133
Epoch 7/100
556/556 [==============================] - 101s 182ms/step - loss: 0.0169 - val_loss: 0.0123
Epoch 8/100
556/556 [==============================] - 100s 180ms/step - loss: 0.0159 - val_loss: 0.0112
Epoch 9/100
556/556 [==============================] - 99s 178ms/step - loss: 0.0158 - val_loss: 0.0116
Epoch 10/100
556/556 [==============================] - 99s 178ms/step

In [8]:
filename = os.listdir("machine-learning-in-science-ii-2023/test_data/test_data") 

df = pd.DataFrame(filename)
df.columns = ["filename"]

df[['file', 'type']] = df.filename.str.split(".", expand = True)
df["file"] = df["file"].astype(str).astype(int)

df.sort_values(by=['file'], inplace=True)
df.reset_index(drop=True, inplace=True)
print(df)


test_datagen = ImageDataGenerator(rescale = 1./255)

test_images = test_datagen.flow_from_dataframe(
    dataframe=df,
    directory="machine-learning-in-science-ii-2023/test_data/test_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)

print(test_images)

prediction = model.predict(test_images)
print(prediction.shape)
import math

a= prediction[:,0]
b= prediction[:,1]
b+=0.5
b= np.floor(b)

submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('Submission.csv')


      filename  file type
0        1.png     1  png
1        2.png     2  png
2        3.png     3  png
3        4.png     4  png
4        5.png     5  png
...        ...   ...  ...
1015  1016.png  1016  png
1016  1017.png  1017  png
1017  1018.png  1018  png
1018  1019.png  1019  png
1019  1020.png  1020  png

[1020 rows x 3 columns]
Found 1020 validated image filenames.
32/32 [==============================] - 4s 106ms/step
(1020, 2)


In [9]:
from sklearn.metrics import classification_report

test_loss = model.evaluate(
    eval_generator,
    verbose=1
)

test_datagen = ImageDataGenerator(rescale = 1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=x_evaluate,
    directory="machine-learning-in-science-ii-2023/training_data/training_data",
    x_col="filename",
    target_size=img_size,
    shuffle=False,
    class_mode=None)


prediction = model.predict(test_generator, verbose =1)

a= prediction[:,0]
b= prediction[:,1]
b+=0.5
b= np.floor(b)

submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('evaluate.csv')



93/93 [==============================] - 8s 88ms/step - loss: 0.0108
Found 2966 validated image filenames.
93/93 [==============================] - 7s 71ms/step


In [10]:
diff = abs(x_evaluate["angle"] - a)

results = pd.DataFrame({"Train":x_evaluate["angle"], "Predicted":a, "Difference":diff})
#results

#results.to_csv('evaluate.csv')

In [11]:
results = pd.DataFrame({"Train":x_evaluate["speed"],"Predicted":b})
#results

#results.to_csv('evaluate.csv')

In [12]:
model.save("model.hdf5")

In [13]:
model_architecture = model.to_json()
model.save_weights("model_weights.h5")

In [18]:
from tensorflow.keras.models import model_from_json
# model2 = model_from_json(model_architecture)
# model2.load_weights("model_weights.h5")

from keras.models import load_model
model2 = load_model('model.h5')


#model2.trainable = True

for layer in model2.layers[:2]:
    layer.trainable = False
    
model2.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 mobilenetv2_1.00_224 (Funct  (None, 8, 10, 1280)      2257984   
 ional)                                                          
                                                                 
 conv2d (Conv2D)             (None, 3, 4, 128)         1474688   
                                                                 
 batch_normalization (BatchN  (None, 3, 4, 128)        512       
 ormalization)                                                   
                                                                 
 flatten (Flatten)           (None, 1536)              0         
                                                                 
 dense (Dense)               (None, 64)                98368     
                                                                 
 batch_normalization_1 (Batc  (None, 64)               2

In [19]:

model2.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.00001), # 1e-5 
    loss='mean_squared_error'
)

es = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    min_delta=0.0005,
    patience=30,
    verbose=1,
    mode='min',
    restore_best_weights=True
)

history = model2.fit(
    train_generator,
    batch_size=batch_size,
    validation_data=val_generator,
    callbacks=[es],
    epochs=300
)


Epoch 1/300
556/556 [==============================] - 111s 193ms/step - loss: 0.0128 - val_loss: 0.0077
Epoch 2/300
556/556 [==============================] - 102s 183ms/step - loss: 0.0124 - val_loss: 0.0075
Epoch 3/300
556/556 [==============================] - 99s 178ms/step - loss: 0.0119 - val_loss: 0.0076
Epoch 4/300
556/556 [==============================] - 99s 178ms/step - loss: 0.0118 - val_loss: 0.0076
Epoch 5/300
556/556 [==============================] - 98s 177ms/step - loss: 0.0120 - val_loss: 0.0077
Epoch 6/300
556/556 [==============================] - 99s 178ms/step - loss: 0.0120 - val_loss: 0.0076
Epoch 7/300
556/556 [==============================] - 99s 178ms/step - loss: 0.0122 - val_loss: 0.0076
Epoch 8/300
556/556 [==============================] - 98s 176ms/step - loss: 0.0121 - val_loss: 0.0076
Epoch 9/300
556/556 [==============================] - 99s 178ms/step - loss: 0.0118 - val_loss: 0.0076
Epoch 10/300
556/556 [==============================] - 99s 17

In [20]:
from sklearn.metrics import classification_report

test_loss = model2.evaluate(
    eval_generator,
    verbose=1
)


93/93 [==============================] - 8s 79ms/step - loss: 0.0076


In [ ]:
# model2.save("model-fine-tuning.hdf5")

In [21]:

prediction = model2.predict(test_images)
print(prediction.shape)
import math

a= prediction[:,0]
b= prediction[:,1]
b+=0.5
b= np.floor(b)

submissiondata={"angle":a , "speed":b} 
df = pd.DataFrame(submissiondata)
df.index += 1 
df.to_csv('Submission-fine-tuning.csv')


32/32 [==============================] - 4s 100ms/step
(1020, 2)


In [17]:
# from keras.models import load_model
# model3 = load_model('model-best.h5')

# df_all = pd.read_csv(r'machine-learning-in-science-ii-2023/training_norm.csv')
# df_all['filename'] = df_all["image_id"].astype(str) + ".png"

# clean_datagen = ImageDataGenerator(rescale = 1./255)

# clean_generator = clean_datagen.flow_from_dataframe(
#     dataframe=df_all,
#     directory="machine-learning-in-science-ii-2023/training_data/training_data",
#     x_col="filename",
#     target_size=img_size,
#     shuffle=False,
#     class_mode=None)


# prediction = model.predict(clean_generator, verbose =1)

# a= prediction[:,0]
# b= prediction[:,1]
# b+=0.5
# b= np.floor(b)


# diff = abs(df_all["angle"] - a)
# results = pd.DataFrame({"image_id":df_all["image_id"], "Actual Angle":df_all["angle"], "Predicted Angle":a, "Angle Difference":diff, "Actual Speed":df_all["speed"], "Predicted Speed":b})

# df = pd.DataFrame(results)
# df.to_csv('clean_results.csv')




Found 14826 validated image filenames.
464/464 [==============================] - 41s 88ms/step
